<a href="https://colab.research.google.com/github/MESIBUN/ds_projects/blob/main/BDA_project_cia_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=593c6d0b2820644969db8f22ebd099f008429a1b378b8f3ad3d151d557c35d27
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier


In [ ]:
# Create a Spark session
spark = SparkSession.builder.appName('DecisionTreeClassifier').getOrCreate()

Workflow


*   Data preparation
*   Feature Engineering
*   Build model
*   Evaluate

#Task
*   Predict if a patient is Hep or not based on parameter
*   The dataset contains laboratry values of blood donors and Hepatitus C patients and demographic values like age







In [ ]:
# Load the data
data = spark.read.csv('hcvdata.csv', header=True, inferSchema=True)


In [ ]:
#Preview the dataset
data.show(5)

+---+-------------+---+---+----+----+----+----+----+-----+----+-----+----+----+
|_c0|     Category|Age|Sex| ALB| ALP| ALT| AST| BIL|  CHE|CHOL| CREA| GGT|PROT|
+---+-------------+---+---+----+----+----+----+----+-----+----+-----+----+----+
|  1|0=Blood Donor| 32|  m|38.5|52.5| 7.7|22.1| 7.5| 6.93|3.23|106.0|12.1|  69|
|  2|0=Blood Donor| 32|  m|38.5|70.3|  18|24.7| 3.9|11.17| 4.8| 74.0|15.6|76.5|
|  3|0=Blood Donor| 32|  m|46.9|74.7|36.2|52.6| 6.1| 8.84| 5.2| 86.0|33.2|79.3|
|  4|0=Blood Donor| 32|  m|43.2|  52|30.6|22.6|18.9| 7.33|4.74| 80.0|33.8|75.7|
|  5|0=Blood Donor| 32|  m|39.2|74.1|32.6|24.8| 9.6| 9.15|4.32| 76.0|29.9|68.7|
+---+-------------+---+---+----+----+----+----+----+-----+----+-----+----+----+
only showing top 5 rows



In [ ]:
 #check fo columns
 print(data.columns)

['_c0', 'Category', 'Age', 'Sex', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT']


In [ ]:
#Rearrange
data = data.select( 'Age', 'Sex', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT','Category')

In [ ]:
 data.show(5)

+---+---+----+----+----+----+----+-----+----+-----+----+----+-------------+
|Age|Sex| ALB| ALP| ALT| AST| BIL|  CHE|CHOL| CREA| GGT|PROT|     Category|
+---+---+----+----+----+----+----+-----+----+-----+----+----+-------------+
| 32|  m|38.5|52.5| 7.7|22.1| 7.5| 6.93|3.23|106.0|12.1|  69|0=Blood Donor|
| 32|  m|38.5|70.3|  18|24.7| 3.9|11.17| 4.8| 74.0|15.6|76.5|0=Blood Donor|
| 32|  m|46.9|74.7|36.2|52.6| 6.1| 8.84| 5.2| 86.0|33.2|79.3|0=Blood Donor|
| 32|  m|43.2|  52|30.6|22.6|18.9| 7.33|4.74| 80.0|33.8|75.7|0=Blood Donor|
| 32|  m|39.2|74.1|32.6|24.8| 9.6| 9.15|4.32| 76.0|29.9|68.7|0=Blood Donor|
+---+---+----+----+----+----+----+-----+----+-----+----+----+-------------+
only showing top 5 rows



In [ ]:
#check th data types
data.dtypes

[('Age', 'int'),
 ('Sex', 'string'),
 ('ALB', 'string'),
 ('ALP', 'string'),
 ('ALT', 'string'),
 ('AST', 'double'),
 ('BIL', 'double'),
 ('CHE', 'double'),
 ('CHOL', 'string'),
 ('CREA', 'double'),
 ('GGT', 'double'),
 ('PROT', 'string'),
 ('Category', 'string')]

In [ ]:
data.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- ALB: string (nullable = true)
 |-- ALP: string (nullable = true)
 |-- ALT: string (nullable = true)
 |-- AST: double (nullable = true)
 |-- BIL: double (nullable = true)
 |-- CHE: double (nullable = true)
 |-- CHOL: string (nullable = true)
 |-- CREA: double (nullable = true)
 |-- GGT: double (nullable = true)
 |-- PROT: string (nullable = true)
 |-- Category: string (nullable = true)



In [ ]:
#Descriptive summary
print(data.describe().show())

+-------+------------------+----+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+-------------+
|summary|               Age| Sex|              ALB|               ALP|               ALT|              AST|               BIL|               CHE|              CHOL|             CREA|              GGT|             PROT|     Category|
+-------+------------------+----+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+-------------+
|  count|               615| 615|              615|               615|               615|              615|               615|               615|               615|              615|              615|              615|          615|
|   mean| 47.40813008130081|NULL|41.62019543973941| 68.2839195979899

In [ ]:
data.groupBy("Category").count().show()

+--------------------+-----+
|            Category|count|
+--------------------+-----+
|       0=Blood Donor|  533|
|         3=Cirrhosis|   30|
|          2=Fibrosis|   21|
|0s=suspect Blood ...|    7|
|         1=Hepatitis|   24|
+--------------------+-----+



In [ ]:
from pyspark.ml.feature import VectorAssembler,StringIndexer

In [ ]:
#unique values for sex
data.select("Sex").distinct().show()

+---+
|Sex|
+---+
|  m|
|  f|
+---+



In [ ]:
#Converting the string into numerical
genderEncode = StringIndexer(inputCol = "Sex",outputCol ="Gender").fit(data)
data = genderEncode.transform(data)

In [ ]:
data.show(2)

+---+---+----+----+---+----+---+-----+----+-----+----+----+-------------+------+
|Age|Sex| ALB| ALP|ALT| AST|BIL|  CHE|CHOL| CREA| GGT|PROT|     Category|Gender|
+---+---+----+----+---+----+---+-----+----+-----+----+----+-------------+------+
| 32|  m|38.5|52.5|7.7|22.1|7.5| 6.93|3.23|106.0|12.1|  69|0=Blood Donor|   0.0|
| 32|  m|38.5|70.3| 18|24.7|3.9|11.17| 4.8| 74.0|15.6|76.5|0=Blood Donor|   0.0|
+---+---+----+----+---+----+---+-----+----+-----+----+----+-------------+------+
only showing top 2 rows



In [ ]:
data.select("Category").distinct().show()

+--------------------+
|            Category|
+--------------------+
|       0=Blood Donor|
|         3=Cirrhosis|
|          2=Fibrosis|
|0s=suspect Blood ...|
|         1=Hepatitis|
+--------------------+



In [ ]:
catEncode = StringIndexer(inputCol = "Category",outputCol ="Target").fit(data)
data = catEncode.transform(data)

from the above process we have encoded values for each categories namely,
*   Blood Donor - 0
*   Cirrhosis - 1
*   Fibrosis - 4
*   suspect Blood donor - 3
*   Hepatitis - 2






In [ ]:
data.select("Target").distinct().show()

+------+
|Target|
+------+
|   0.0|
|   1.0|
|   4.0|
|   3.0|
|   2.0|
+------+



In [ ]:
data.columns

['Age',
 'Sex',
 'ALB',
 'ALP',
 'ALT',
 'AST',
 'BIL',
 'CHE',
 'CHOL',
 'CREA',
 'GGT',
 'PROT',
 'Category',
 'Gender',
 'Category_index',
 'Target']

In [ ]:
df = data.select('Age','ALB','ALP','ALT', 'AST', 'BIL', 'CHE','CHOL','CREA','GGT','PROT', 'Gender','Target')

In [ ]:
df.show()

+---+----+----+----+----+----+-----+----+-----+----+----+------+------+
|Age| ALB| ALP| ALT| AST| BIL|  CHE|CHOL| CREA| GGT|PROT|Gender|Target|
+---+----+----+----+----+----+-----+----+-----+----+----+------+------+
| 32|38.5|52.5| 7.7|22.1| 7.5| 6.93|3.23|106.0|12.1|  69|   0.0|   0.0|
| 32|38.5|70.3|  18|24.7| 3.9|11.17| 4.8| 74.0|15.6|76.5|   0.0|   0.0|
| 32|46.9|74.7|36.2|52.6| 6.1| 8.84| 5.2| 86.0|33.2|79.3|   0.0|   0.0|
| 32|43.2|  52|30.6|22.6|18.9| 7.33|4.74| 80.0|33.8|75.7|   0.0|   0.0|
| 32|39.2|74.1|32.6|24.8| 9.6| 9.15|4.32| 76.0|29.9|68.7|   0.0|   0.0|
| 32|41.6|43.3|18.5|19.7|12.3| 9.92|6.05|111.0|91.0|  74|   0.0|   0.0|
| 32|46.3|41.3|17.5|17.8| 8.5| 7.01|4.79| 70.0|16.9|74.5|   0.0|   0.0|
| 32|42.2|41.9|35.8|31.1|16.1| 5.82| 4.6|109.0|21.5|67.1|   0.0|   0.0|
| 32|50.9|65.5|23.2|21.2| 6.9| 8.69| 4.1| 83.0|13.7|71.3|   0.0|   0.0|
| 32|42.4|86.3|20.3|20.0|35.2| 5.46|4.45| 81.0|15.9|69.9|   0.0|   0.0|
| 32|44.3|52.3|21.7|22.4|17.2| 4.15|3.57| 78.0|24.1|75.4|   0.0|

In [ ]:
df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- ALB: string (nullable = true)
 |-- ALP: string (nullable = true)
 |-- ALT: string (nullable = true)
 |-- AST: double (nullable = true)
 |-- BIL: double (nullable = true)
 |-- CHE: double (nullable = true)
 |-- CHOL: string (nullable = true)
 |-- CREA: double (nullable = true)
 |-- GGT: double (nullable = true)
 |-- PROT: string (nullable = true)
 |-- Gender: double (nullable = false)
 |-- Target: double (nullable = false)



In [ ]:
# since the dtypes are in string we convert it into "toPandas" and data type as "float"

df =df.toPandas().replace("NA",0).astype(float)

In [ ]:
type(df)

pandas.core.frame.DataFrame

In [ ]:
type(data)

pyspark.sql.dataframe.DataFrame

In [ ]:
new_df = spark.createDataFrame(df)

In [ ]:
new_df.printSchema()

root
 |-- Age: double (nullable = true)
 |-- ALB: double (nullable = true)
 |-- ALP: double (nullable = true)
 |-- ALT: double (nullable = true)
 |-- AST: double (nullable = true)
 |-- BIL: double (nullable = true)
 |-- CHE: double (nullable = true)
 |-- CHOL: double (nullable = true)
 |-- CREA: double (nullable = true)
 |-- GGT: double (nullable = true)
 |-- PROT: double (nullable = true)
 |-- Gender: double (nullable = true)
 |-- Target: double (nullable = true)



In [ ]:
new_df.show()

+----+----+----+----+----+----+-----+----+-----+----+----+------+------+
| Age| ALB| ALP| ALT| AST| BIL|  CHE|CHOL| CREA| GGT|PROT|Gender|Target|
+----+----+----+----+----+----+-----+----+-----+----+----+------+------+
|32.0|38.5|52.5| 7.7|22.1| 7.5| 6.93|3.23|106.0|12.1|69.0|   0.0|   0.0|
|32.0|38.5|70.3|18.0|24.7| 3.9|11.17| 4.8| 74.0|15.6|76.5|   0.0|   0.0|
|32.0|46.9|74.7|36.2|52.6| 6.1| 8.84| 5.2| 86.0|33.2|79.3|   0.0|   0.0|
|32.0|43.2|52.0|30.6|22.6|18.9| 7.33|4.74| 80.0|33.8|75.7|   0.0|   0.0|
|32.0|39.2|74.1|32.6|24.8| 9.6| 9.15|4.32| 76.0|29.9|68.7|   0.0|   0.0|
|32.0|41.6|43.3|18.5|19.7|12.3| 9.92|6.05|111.0|91.0|74.0|   0.0|   0.0|
|32.0|46.3|41.3|17.5|17.8| 8.5| 7.01|4.79| 70.0|16.9|74.5|   0.0|   0.0|
|32.0|42.2|41.9|35.8|31.1|16.1| 5.82| 4.6|109.0|21.5|67.1|   0.0|   0.0|
|32.0|50.9|65.5|23.2|21.2| 6.9| 8.69| 4.1| 83.0|13.7|71.3|   0.0|   0.0|
|32.0|42.4|86.3|20.3|20.0|35.2| 5.46|4.45| 81.0|15.9|69.9|   0.0|   0.0|
|32.0|44.3|52.3|21.7|22.4|17.2| 4.15|3.57| 78.0|24.

In [ ]:
#VectorAssembler
req_features = ['Age','ALB','ALP','ALT','AST','BIL','CHE','CHOL','CREA','GGT','PROT','Gender']
vec_assemb = VectorAssembler(inputCols=req_features,outputCol ="features")


In [ ]:
vec_df = vec_assemb.transform(new_df)

In [ ]:
vec_df.show(4)

+----+----+----+----+----+----+-----+----+-----+----+----+------+------+--------------------+
| Age| ALB| ALP| ALT| AST| BIL|  CHE|CHOL| CREA| GGT|PROT|Gender|Target|            features|
+----+----+----+----+----+----+-----+----+-----+----+----+------+------+--------------------+
|32.0|38.5|52.5| 7.7|22.1| 7.5| 6.93|3.23|106.0|12.1|69.0|   0.0|   0.0|[32.0,38.5,52.5,7...|
|32.0|38.5|70.3|18.0|24.7| 3.9|11.17| 4.8| 74.0|15.6|76.5|   0.0|   0.0|[32.0,38.5,70.3,1...|
|32.0|46.9|74.7|36.2|52.6| 6.1| 8.84| 5.2| 86.0|33.2|79.3|   0.0|   0.0|[32.0,46.9,74.7,3...|
|32.0|43.2|52.0|30.6|22.6|18.9| 7.33|4.74| 80.0|33.8|75.7|   0.0|   0.0|[32.0,43.2,52.0,3...|
+----+----+----+----+----+----+-----+----+-----+----+----+------+------+--------------------+
only showing top 4 rows



Train test split

In [ ]:
train_df,test_df = vec_df.randomSplit([0.7,0.3])

In [ ]:
train_df.count()

419

In [ ]:
# Model building
from pyspark.ml.classification import LogisticRegression,DecisionTreeClassifier

In [ ]:
lr = LogisticRegression(featuresCol ="features",labelCol="Target")

In [ ]:
lr_model = lr.fit(train_df)

In [97]:
y_pred = lr_model.transform(test_df)


In [ ]:
y_pred.show()

+----+----+-----+-----+----+----+-----+----+-----+----+----+------+------+--------------------+--------------------+--------------------+----------+
| Age| ALB|  ALP|  ALT| AST| BIL|  CHE|CHOL| CREA| GGT|PROT|Gender|Target|            features|       rawPrediction|         probability|prediction|
+----+----+-----+-----+----+----+-----+----+-----+----+----+------+------+--------------------+--------------------+--------------------+----------+
|32.0|42.2| 41.9| 35.8|31.1|16.1| 5.82| 4.6|109.0|21.5|67.1|   0.0|   0.0|[32.0,42.2,41.9,3...|[254.439254377550...|[1.0,2.0550534104...|       0.0|
|32.0|46.3| 41.3| 17.5|17.8| 8.5| 7.01|4.79| 70.0|16.9|74.5|   0.0|   0.0|[32.0,46.3,41.3,1...|[252.602279747994...|[1.0,4.1245379117...|       0.0|
|32.0|46.9| 74.7| 36.2|52.6| 6.1| 8.84| 5.2| 86.0|33.2|79.3|   0.0|   0.0|[32.0,46.9,74.7,3...|[243.818134937929...|[1.0,1.3289519517...|       0.0|
|33.0|36.3| 78.6| 23.6|22.0| 7.0| 8.56|5.38| 78.0|19.4|68.7|   0.0|   0.0|[33.0,36.3,78.6,2...|[460.711025

In [ ]:
print(y_pred.columns)

['Age', 'ALB', 'ALP', 'ALT', 'AST', 'BIL', 'CHE', 'CHOL', 'CREA', 'GGT', 'PROT', 'Gender', 'Target', 'features', 'rawPrediction', 'probability', 'prediction']


In [ ]:
y_pred.select('Target','rawPrediction', 'probability', 'prediction').show()

+------+--------------------+--------------------+----------+
|Target|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+----------+
|   0.0|[254.439254377550...|[1.0,2.0550534104...|       0.0|
|   0.0|[252.602279747994...|[1.0,4.1245379117...|       0.0|
|   0.0|[243.818134937929...|[1.0,1.3289519517...|       0.0|
|   0.0|[460.711025052271...|[1.0,3.7302369505...|       0.0|
|   0.0|[241.631950491873...|[1.0,8.8667474143...|       0.0|
|   0.0|[465.940304168253...|[1.0,1.0026957232...|       0.0|
|   0.0|[286.870845909243...|[1.0,5.0443356538...|       0.0|
|   0.0|[344.751589316392...|[1.0,3.6725691488...|       0.0|
|   0.0|[469.140852972760...|[1.0,1.5846757037...|       0.0|
|   0.0|[506.493971305243...|[1.0,0.0,3.087720...|       0.0|
|   0.0|[282.529214254763...|[1.0,1.3586829318...|       0.0|
|   0.0|[315.909197014438...|[1.0,1.7654862487...|       0.0|
|   0.0|[400.471493383307...|[1.0,1.1496485541...|       0.0|
|   0.0|

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
mult_eval =MulticlassClassificationEvaluator(labelCol="Target",metricName="accuracy")

In [ ]:
mult_eval.evaluate(y_pred)

0.9030612244897959

In [95]:
dc = DecisionTreeClassifier(featuresCol ="features",labelCol="Target")

In [102]:
dc_model = dc.fit(train_df)
y_pred = dc_model.transform(test_df)


In [103]:
y_pred.show()

+----+----+-----+-----+----+----+-----+----+-----+----+----+------+------+--------------------+--------------------+--------------------+----------+
| Age| ALB|  ALP|  ALT| AST| BIL|  CHE|CHOL| CREA| GGT|PROT|Gender|Target|            features|       rawPrediction|         probability|prediction|
+----+----+-----+-----+----+----+-----+----+-----+----+----+------+------+--------------------+--------------------+--------------------+----------+
|32.0|42.2| 41.9| 35.8|31.1|16.1| 5.82| 4.6|109.0|21.5|67.1|   0.0|   0.0|[32.0,42.2,41.9,3...|[353.0,0.0,4.0,1....|[0.98328690807799...|       0.0|
|32.0|46.3| 41.3| 17.5|17.8| 8.5| 7.01|4.79| 70.0|16.9|74.5|   0.0|   0.0|[32.0,46.3,41.3,1...|[353.0,0.0,4.0,1....|[0.98328690807799...|       0.0|
|32.0|46.9| 74.7| 36.2|52.6| 6.1| 8.84| 5.2| 86.0|33.2|79.3|   0.0|   0.0|[32.0,46.9,74.7,3...|[8.0,0.0,0.0,0.0,...|[1.0,0.0,0.0,0.0,...|       0.0|
|33.0|36.3| 78.6| 23.6|22.0| 7.0| 8.56|5.38| 78.0|19.4|68.7|   0.0|   0.0|[33.0,36.3,78.6,2...|[353.0,0.0,

In [99]:
y_pred.select('Target','rawPrediction', 'probability', 'prediction').show()

+------+--------------------+--------------------+----------+
|Target|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+----------+
|   0.0|[254.439254377550...|[1.0,2.0550534104...|       0.0|
|   0.0|[252.602279747994...|[1.0,4.1245379117...|       0.0|
|   0.0|[243.818134937929...|[1.0,1.3289519517...|       0.0|
|   0.0|[460.711025052271...|[1.0,3.7302369505...|       0.0|
|   0.0|[241.631950491873...|[1.0,8.8667474143...|       0.0|
|   0.0|[465.940304168253...|[1.0,1.0026957232...|       0.0|
|   0.0|[286.870845909243...|[1.0,5.0443356538...|       0.0|
|   0.0|[344.751589316392...|[1.0,3.6725691488...|       0.0|
|   0.0|[469.140852972760...|[1.0,1.5846757037...|       0.0|
|   0.0|[506.493971305243...|[1.0,0.0,3.087720...|       0.0|
|   0.0|[282.529214254763...|[1.0,1.3586829318...|       0.0|
|   0.0|[315.909197014438...|[1.0,1.7654862487...|       0.0|
|   0.0|[400.471493383307...|[1.0,1.1496485541...|       0.0|
|   0.0|

In [104]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
mult_eval =MulticlassClassificationEvaluator(labelCol="Target",metricName="accuracy")

In [105]:
mult_eval.evaluate(y_pred)

0.9132653061224489